In [1]:
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import l2beat_utils as ltwo
sys.path.pop()

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# # Get token classification
# # https://dune.com/queries/3852319
token_df = d.get_dune_data(query_id = 3852319,
    name = "tokens_by_type_base_opm",
    performance = 'large'#,
    # num_hours_to_rerun=0
)

token_df.sample(5)

2024-06-21 20:02:11.837 | INFO     | duneapi_utils:get_dune_data:58 - Results available at https://dune.com/queries/3852319
2024-06-21 20:02:20.274 | INFO     | duneapi_utils:get_dune_data:105 - ✨ Results saved as csv_outputs/tokens_by_type_base_opm.csv, with 314585 rows and 8 columns.


,blockchain,chain_id,factory_address,l1_token,symbol,token_address,token_classification,last_updated
151787,base,8453,None,None,DIAMOND,0x3ae675993e76fffdb2491ee8b0d4e81fe19d5dce,Not Proxy or Factory,2024-06-21 23:49:59.603507
5760,base,8453,None,None,NEGED,0x4815a86efa4fe9c1891f4259b5efabf8552ba701,Not Proxy or Factory,2024-06-21 23:49:59.603507
156288,base,8453,None,None,ICPB,0xf8fb41dd5e27a0a8f90fd35960eb54bf193e8a89,Not Proxy or Factory,2024-06-21 23:49:59.603507
89019,base,8453,None,None,GUSH,0xe600e7ecb80e8ca58fc6230df0105cfa150b1922,Not Proxy or Factory,2024-06-21 23:49:59.603507
87134,base,8453,None,None,ZEUS,0x29d326c7327e892b87857d7294c6d5e58c49ee01,Not Proxy or Factory,2024-06-21 23:49:59.603507


In [3]:
# Get Lastest L2B aoc
l2b_df = ltwo.get_daily_aoc_by_token()
l2b_df_spc = l2b_df[l2b_df['project'].isin(['optimism','base'])]
l2b_df_spc = l2b_df_spc.rename(columns={'project':'blockchain'})

l2b_df_spc = l2b_df_spc[['dt','blockchain','token_type','asset_id','address','usd_value']]
l2b_df_spc['address'] = l2b_df_spc['address'].str.lower()

l2b_df_spc.sample(5)

,dt,blockchain,token_type,asset_id,address,usd_value
790,2024-06-22,optimism,canonical,link-chainlink-token,0x514910771af9ca656af840dff83e8264ecf986ca,6.105947e+06
781,2024-06-22,optimism,canonical,hair-hairdao-token,0x9ce115f0341ae5dabc8b477b74e83db2018a6f42,2.683699e+06
740,2024-06-22,optimism,canonical,rpl-rocket-pool-protocol,0xd33526068d116ce69f19a9ee46f0bd304f21a51f,1.290014e+04
262,2024-06-22,base,native,base:toshi-toshi,0xac1bd2486aaf3b5c0fc3fd868558b082a531b2b4,1.023253e+08
225,2024-06-22,base,canonical,usdt-tether-usd,0xdac17f958d2ee523a2206206994597c13d831ec7,2.512475e+06


In [4]:
cols = ['blockchain','token_address','l1_token','token_classification','symbol']

token_if_l1_df = token_df[cols].rename(columns={'l1_token':'address'})
token_if_l1_df['l1_address'] = token_if_l1_df['address']
token_if_l2_df = token_df[cols].rename(columns={'token_address':'address'})
token_if_l2_df['l2_address'] = token_if_l2_df['address']

In [5]:
df_cols = ['blockchain','address','token_classification']
token_if_l1_df = token_if_l1_df[~token_if_l1_df['address'].isna()][df_cols].drop_duplicates()
token_if_l2_df = token_if_l2_df[~token_if_l2_df['address'].isna()][df_cols].drop_duplicates()

token_if_l2_df.sample(5)

,blockchain,address,token_classification
151934,base,0xb9b7786f2a8abca0c1eceee120aa1c8e506cadbb,Not Proxy or Factory
7812,base,0x0e9d3a85f484bdecdb33f07ff298a2d1bcd18055,Not Proxy or Factory
185694,base,0x490bd628fac3cd04ea13e1b75d5c7110ddf8dde2,Not Proxy or Factory
190633,base,0x801a22aeefe03c9961feff6c9c2a6f33de97e14d,Not Proxy or Factory
208768,base,0x50ddae5d6aecfdfe7a702e90850ece5834b44f53,Not Proxy or Factory


In [6]:
#L2 Mapping preferred
unified_df = l2b_df_spc.merge(token_if_l2_df
                              , on=['blockchain','address'], how='left')
#L1 Mapping fallback
unified_df = unified_df.merge(token_if_l1_df
                              , on=['blockchain','address'], how='left')


In [7]:
unified_df['token_classification'] = unified_df['token_classification_x'].fillna(unified_df['token_classification_y']).fillna('Unknown')

unified_df = unified_df.drop(columns=['token_classification_x', 'token_classification_y'])

In [8]:
unified_df['token_classification'] = unified_df.apply(
    lambda row: 'Native' if row['asset_id'] == 'eth-ether' else row['token_classification'], axis=1
)
# Override due to bad Base events
unified_df['token_classification'] = unified_df.apply(
    lambda row: 'Proxy' if row['asset_id'] == 'base:usdc-usd-coin' else row['token_classification'], axis=1
)
# Override OP predeploy
unified_df['token_classification'] = unified_df.apply(
    lambda row: 'Predeploy' if row['asset_id'] == 'op-optimism' else row['token_classification'], axis=1
)

In [9]:
# Define the priority order for token_classification
priority = {'Proxy': 1, 'Not Proxy or Factory': 2, 'Immutable Factory Created': 3}

# Add a priority column to the DataFrame based on the token_classification
unified_df['priority'] = unified_df['token_classification'].map(priority)

# Sort the DataFrame by the priority column
unified_df = unified_df.sort_values(by='priority')

# Drop duplicates based on the unique columns, keeping the first occurrence (which has the highest priority)
unique_columns = ['dt', 'blockchain', 'token_type', 'asset_id', 'address']
unified_df = unified_df.drop_duplicates(subset=unique_columns, keep='first')

# Drop the priority column as it's no longer needed
unified_df = unified_df.drop(columns=['priority'])

In [10]:
unified_df.to_csv('csv_outputs/aoc_by_chain_token_classification.csv')

In [11]:
unified_df[unified_df['address']=='0x833589fcd6edb6e08f4c7c32d4f71b54bda02913']#.sample(5)

,dt,blockchain,token_type,asset_id,address,usd_value,token_classification
44,2024-06-22,base,native,base:usdc-usd-coin,0x833589fcd6edb6e08f4c7c32d4f71b54bda02913,3.056719e+09,Proxy
